In [1]:
## uploading and downloading LULC files for soc covariates

import os
import zipfile
import rasterio
import numpy as np
import requests
from tqdm import tqdm



In [ ]:
# ===========================================================
# CONFIGURATION
# ===========================================================

# HYDE 3.2.1 Baseline FTP
BASE_URL = "https://geo.vu.nl/~hyde/download/HYDE3.2.1/Baseline/zip/"
GENERAL_URL = "https://geo.vu.nl/~hyde/download/HYDE3.2.1/General%20files/"

YEARS = [1950, 1960, 1970, 1975]
VARIABLES = ["cropland", "grazing", "rangeland", "pasture"]
ISO_CODE = 24  # Angola
OUTDIR = "hyde_angola_outputs"
os.makedirs(OUTDIR, exist_ok=True)

In [ ]:
# ===========================================================
# HELPER FUNCTIONS
# ===========================================================

def download_file(url, dest):
    """Download file with progress bar"""
    if os.path.exists(dest):
        print(f"✅ Already downloaded: {os.path.basename(dest)}")
        return
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get("content-length", 0))
    with open(dest, "wb") as file, tqdm(
        desc=os.path.basename(dest), total=total, unit="B", unit_scale=True
    ) as bar:
        for chunk in resp.iter_content(chunk_size=8192):
            file.write(chunk)
            bar.update(len(chunk))

def extract_zip(file_path, extract_dir):
    with zipfile.ZipFile(file_path, "r") as zip_ref:
        zip_ref.extractall(extract_dir)

# ===========================================================
# DOWNLOAD GENERAL FILES
# ===========================================================
print("\n📥 Downloading country mask and area grid...")
for f in ["iso_cr.asc", "garea_cr.asc"]:
    download_file(GENERAL_URL + f, os.path.join(OUTDIR, f))

# Load mask and area rasters
print("📖 Reading country mask and grid area rasters...")
with rasterio.open(os.path.join(OUTDIR, "iso_cr.asc")) as src_iso:
    iso = src_iso.read(1)
    profile = src_iso.profile
with rasterio.open(os.path.join(OUTDIR, "garea_cr.asc")) as src_area:
    garea = src_area.read(1)

# ===========================================================
# PROCESS EACH YEAR
# ===========================================================
for year in YEARS:
    zip_filename = f"{year}AD_lu.zip"
    zip_path = os.path.join(OUTDIR, zip_filename)
    print(f"\n📥 Downloading HYDE data for {year}...")
    download_file(BASE_URL + zip_filename, zip_path)

    extract_dir = os.path.join(OUTDIR, f"{year}_unzipped")
    if not os.path.exists(extract_dir):
        extract_zip(zip_path, extract_dir)

    for var in VARIABLES:
        asc_path = os.path.join(extract_dir, f"{var}{year}.asc")
        if not os.path.exists(asc_path):
            print(f"⚠️ {var}{year}.asc not found in archive")
            continue

        with rasterio.open(asc_path) as src:
            data = src.read(1).astype(float)

        # Mask for Angola
        masked = np.where(iso == ISO_CODE, data, np.nan)

        # Convert to fractional coverage (0–1)
        frac = np.divide(masked, garea, where=(garea > 0))
        frac[np.isnan(frac)] = src.nodata if src.nodata is not None else -9999

        # Save as GeoTIFF
        out_path = os.path.join(OUTDIR, f"angola_{var}_{year}_frac.tif")
        profile.update(dtype="float32", nodata=-9999)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(frac.astype("float32"), 1)

        print(f"✅ Saved: {out_path}")

print("\n🎉 Done! Fractional LULC rasters saved in:", OUTDIR)
